In [1]:
#Import libreries
import plotly.express as px
from jupyter_dash import JupyterDash
#import dash_core_components as dcc #deprecated
from dash import dcc
#import dash_html_components as html #deprecated
from dash import html
from dash.dependencies import Input, Output

import sqlite3
import pandas as pd

### Scryfall Data

In [2]:
#Generates (or creates if it doesn't exist) the db connection.
conn = sqlite3.connect("sqlite_db/mtg_cards.db")

#SQL query for testing
sql_result = conn.execute(
    """ SELECT
            scryfall_cards.card_id,
            scryfall_cards.name,
            scryfall_cards.set_name,
            scryfall_cards.reserved,
            scryfall_cards.rarity,
            scryfall_cards.normal,
            scryfall_daily_prices.usd,
            scryfall_daily_prices.usd_foil,
            scryfall_daily_prices.date_time
        FROM 
            scryfall_cards
        LEFT JOIN 
            scryfall_daily_prices
            ON 
                scryfall_cards.card_id = scryfall_daily_prices.card_id
;""")


#Obtains the table column names because sqlite query returns only the data.
colums_names = [column[0] for column in sql_result.description]
#Creates a pandas dataframe qith the query data and the column names.
sql_df = pd.DataFrame.from_records(data = sql_result.fetchall(), columns = colums_names)

#Close connection
conn.close()

print(sql_df.head(5))

                                card_id         name     set_name  reserved  \
0  0000579f-7b35-4ed3-b44c-db2a538066fe  Fury Sliver  Time Spiral         0   
1  0000579f-7b35-4ed3-b44c-db2a538066fe  Fury Sliver  Time Spiral         0   
2  0000579f-7b35-4ed3-b44c-db2a538066fe  Fury Sliver  Time Spiral         0   
3  0000579f-7b35-4ed3-b44c-db2a538066fe  Fury Sliver  Time Spiral         0   
4  0000579f-7b35-4ed3-b44c-db2a538066fe  Fury Sliver  Time Spiral         0   

     rarity                                             normal   usd usd_foil  \
0  uncommon  https://c1.scryfall.com/file/scryfall-cards/no...  0.35     7.15   
1  uncommon  https://c1.scryfall.com/file/scryfall-cards/no...  0.35     7.15   
2  uncommon  https://c1.scryfall.com/file/scryfall-cards/no...  0.35     7.15   
3  uncommon  https://c1.scryfall.com/file/scryfall-cards/no...  0.35     7.15   
4  uncommon  https://c1.scryfall.com/file/scryfall-cards/no...  0.35     7.15   

  date_time  
0  20220417  
1  2022041

### Interactive Dash

In [3]:
#Build app
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash MTG Demo"),
    
    dcc.Dropdown(id="select-set",
                 options=[
                     {'label': c, 'value': c} 
                     for c in sql_df['set_name'].unique()],
                 multi=False,
                 value="Urza's Destiny",
                 style={'width': "40%"}
                 ),
    
    dcc.Dropdown(id="select-card",
                 options=[
                     {'label': c, 'value': c} 
                     for c in sql_df['name'].unique()],
                 multi=False,
                 value="Replenish",
                 style={'width': "40%"}
                 ),
    
    dcc.Graph(id='graph')
])

#Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("select-set", "value"), 
     Input("select-card", "value")]
)
def update_figure(set_selected, card_selected):
    dff = sql_df.copy()
    dff = dff[(dff.set_name == set_selected) & (dff.name == card_selected)]
    
    fig = px.scatter(dff, x="date_time", y="usd", 
                     color_continuous_scale=px.colors.sequential.YlOrRd,
                     render_mode="webgl", title="Tips")
    fig.add_scatter(x=dff["date_time"], y=dff["usd_foil"])
    
    return fig

#Run app and display result inline in the notebook
#app.run_server(mode='inline')
app.run_server(mode='external')
#app.run_server(mode='jupyterlab')

Dash app running on http://127.0.0.1:8050/
